In [5]:
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Función para cargar y preprocesar la imagen
def preprocess_image(image_path, target_size=(64, 64)):
    image = Image.open(image_path).convert('RGB')  # Convertir a RGB
    low_res_image = image.resize(target_size, Image.Resampling.LANCZOS)  # Redimensionar a 64x64
    input_array = np.array(low_res_image) / 255.0  # Normalizar
    input_array = input_array.astype(np.float32)  # Convertir a float32
    input_array = input_array.reshape(1, target_size[0], target_size[1], 3)  # Ajustar forma para el modelo
    return input_array, low_res_image

# Función para cargar el modelo .pb
def load_pb_model(model_pb_path):
    # Load the protobuf file from the disk and parse it to retrieve the unserialized graph_def
    with tf.io.gfile.GFile(model_pb_path, "rb") as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, import the graph_def into a new Graph and return it
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name="")
    return graph

# Función para realizar predicciones con el modelo .pb
def predict_with_pb_model(graph, input_image):
    with tf.compat.v1.Session(graph=graph) as sess:
        input_tensor = graph.get_tensor_by_name("IteratorGetNext:0")  # Ajustar según el nombre del tensor de entrada
        output_tensor = graph.get_tensor_by_name("NHWC_output:0")  # Ajustar según el nombre del tensor de salida
        predictions = sess.run(output_tensor, feed_dict={input_tensor: input_image})
    return predictions

# Función para postprocesar y mostrar la imagen
def show_comparison(input_image, predicted_image_trained, predicted_image_pretrained):
    predicted_image_trained = predicted_image_trained.reshape(256, 256, 3)
    predicted_image_pretrained = predicted_image_pretrained.reshape(256, 256, 3)
    
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.title('Low Resolution')
    plt.imshow(input_image)
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.title('Predicted High Resolution (Trained)')
    plt.imshow(predicted_image_trained)
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.title('Predicted High Resolution (Pretrained)')
    plt.imshow(predicted_image_pretrained)
    plt.axis('off')

    plt.show()

# Ruta a la nueva imagen
new_image_path = r"C:\Users\lina-\Pictures\Camera Roll\WIN_20240605_02_46_44_Pro.jpg"

# Preprocesar la nueva imagen
input_array, low_res_image = preprocess_image(new_image_path)

# Cargar el modelo entrenado guardado en formato .h5
trained_model = load_model('fsrcnn_model_color.h5', compile=False)
trained_model.compile(optimizer='adam', loss='mean_squared_error')

# Hacer predicción con el modelo entrenado
predicted_array_trained = trained_model.predict(input_array)

# Cargar el modelo preentrenado en formato .pb
model_pb_path = r'C:\Users\lina-\Documents\ProyectoComputacion\Proyecto\Codigos\FSRCNN_x4.pb'
pretrained_model = load_pb_model(model_pb_path)

# Hacer predicción con el modelo preentrenado
predicted_array_pretrained = predict_with_pb_model(pretrained_model, input_array)

# Mostrar las imágenes comparadas
show_comparison(low_res_image, predicted_array_trained[0], predicted_array_pretrained[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step



InvalidArgumentError: Graph execution error:

Detected at node 'add' defined at (most recent call last):
Node: 'add'
output depth must be evenly divisible by number of groups: 56 vs 3
	 [[{{node add}}]]

Original stack trace for 'add':
